In [44]:
from datetime import datetime
from app import crud, models, schemas
from app.db.session import SessionLocal
from sqlalchemy.orm import Session
from sqlalchemy.sql import func
from app.services.controlling import ControllingCalendar

db = SessionLocal()
calendar = ControllingCalendar(year=2020, monthrange=[10,12]).get()
start = calendar['range']['start']
end = calendar['range']['end']

def get_clean_discharge(
        _db: Session = db, *, start: datetime = None, end: datetime = None,
        store_internal_id: int = None, owner_id: int = None
) -> float:
    """
    TO AVOID MULTIPLE COUNTED DISCHARGES
    WE ONLY ACCEPT ONE DISCHARGE PER CONTRACT
    MEANS WE VE TO EXCLUDE ALL OTHER ROWS WITH THE SAME CONTRACT NR
    """
    exclude_contracts: t.List[int] = []
    discharges: t.List[float] = []
    total_discharges: float = 0.00    
    #  FOR BETTER SQL PERFORMANCE WE ENCLOSE OUR QUERY: EXCLUDE ALL WITH DISCHARGE IS NULL;
    claims = crud.claim.get_query(_db)
    if start and end:
        claims = claims.filter(models.Claim.created_at.between(start, end))  # noqa
    if store_internal_id:
        claims = claims.filter(models.Claim.store_internal_id == store_internal_id)  # noqa
    if owner_id:
        claims = claims.filter(models.Claim.owner_id == owner_id)  # noqa
    claims = claims.filter(models.Claim.discharge.isnot(None))  # noqa
    claims: t.List[models.Claim] = claims.all()  # noqa

    for claim in claims:
        if claim.contract_nr not in exclude_contracts:
            exclude_contracts.append(claim.contract_nr)
            discharges.append(claim.discharge)
    total_discharges = sum(discharges) if len(discharges) != 0 else 0.00
    total_discharges_round: float = round(total_discharges, 2) 
    return total_discharges_round

In [47]:
def get_claim_bill(
        _db: Session = db, *, start: datetime = None, end: datetime = None,
        store_internal_id: int = None, owner_id: int = None, calculation: str = "avg"
):
    query = crud.claim.get_query(_db)
    if start and end:
        query = query.filter(models.Claim.created_at.between(start, end))
    if store_internal_id:
        query = query.filter(models.Claim.store_internal_id == store_internal_id)
    if owner_id:
        query = query.filter(models.Claim.owner_id == owner_id)
    bill = query.with_entities(getattr(func, calculation)(models.Claim.bill).label("_bill")).first()
    bill_round = round(bill[0], 2) if bill[0] is not None else 0.00
    
    return bill_round

In [22]:
y = get_clean_discharge(start=start, end=end, store_internal_id=188)

In [48]:
for user in crud.user.get_query(db).all():
    print(user.last_name)
    user_average_claim_bill = get_claim_bill(start=start, end=end, owner_id=user.id)
    user_claim_total = get_claim_bill(start=start, end=end, owner_id=user.id, calculation="sum")
    user_discharge = get_clean_discharge(start=start, end=end, owner_id=user.id)
    print(user_average_claim_bill, user_discharge, ".. total bill:", user_claim_total)

Scharbatke
0.0 0.0 .. total bill: 0.0
Esme
92.65 -718.4 .. total bill: 2038.3
Schmidt
91.25 -610.57 .. total bill: 6296.45
Neuendorf
115.15 -2125.84 .. total bill: 6794.02
Cara
92.43 -964.3 .. total bill: 5730.96
Maiberg
0.0 0.0 .. total bill: 0.0
linux
0.0 0.0 .. total bill: 0.0


In [59]:
for store in crud.store.get_query(db).order_by(models.Store.name).all():
    store_average_claim_bill = get_claim_bill(start=start, end=end, store_internal_id=store.internal_id)
    store_claim_total_bill = get_claim_bill(start=start, end=end, store_internal_id=store.internal_id, calculation="sum")
    store_discharge_anno = get_clean_discharge(start=start, end=end, store_internal_id=store.internal_id)
    store_tickets_total = get_tickets_count(start=start, end=end, store_internal_id=store.internal_id)
    store_tickets_commission = get_tickets_count(start=start, end=end, store_internal_id=store.internal_id, kind=models.TicketKind.COMMISSION)
    store_tickets_claim = get_tickets_count(start=start, end=end, store_internal_id=store.internal_id, kind=models.TicketKind.CLAIM)

    if store_average_claim_bill > 0 or store_discharge_anno > 0:
        print(store.name)
        print(store_average_claim_bill, store_discharge_anno)
        print("bill total...", store_claim_total_bill)
        print("tickets total...", store_tickets_total, "...commission", store_tickets_commission, "...claims", store_tickets_claim)

AMBERG
99.22 0.0
bill total... 1091.44
tickets total... 90 ...commission 82 ...claims 8
BÖBLINGEN
95.54 -1527.24
bill total... 2006.34
tickets total... 114 ...commission 98 ...claims 16
BONN
105.07 0.0
bill total... 945.63
tickets total... 145 ...commission 132 ...claims 13
BOTTROP
250.51 -392.81
bill total... 1753.58
tickets total... 65 ...commission 47 ...claims 18
ECHING
127.36 0.0
bill total... 2165.12
tickets total... 161 ...commission 144 ...claims 17
FRIEDRICHSHAFEN
100.56 0.0
bill total... 1005.65
tickets total... 151 ...commission 111 ...claims 40
GÖPPINGEN
64.93 -582.98
bill total... 2597.33
tickets total... 181 ...commission 120 ...claims 61
GÖRGESHAUSEN
86.03 0.0
bill total... 1032.39
tickets total... 79 ...commission 68 ...claims 11
HOMBURG
98.88 -56.86
bill total... 494.4
tickets total... 87 ...commission 83 ...claims 4
KAISERSLAUTERN
108.72 -1565.22
bill total... 3805.31
tickets total... 181 ...commission 142 ...claims 39
KOBLENZ
34.77 0.0
bill total... 382.52
tickets to

In [55]:
def get_tickets_count(
        _db: Session = db, *, start: datetime = None, end: datetime = None,
        store_internal_id: int = None, owner_id: int = None, 
        kind: models.TicketKind = None, status: models.TicketStatus = None
):
    query = crud.ticket.get_query(_db)
    if start and end:
        query = query.filter(models.Ticket.created_at.between(start, end))
    if store_internal_id:
        query = query.filter(models.Ticket.store_internal_id == store_internal_id)
    if owner_id:
        query = query.filter(models.Ticket.owner_id == owner_id)
    if kind:
        query = query.filter(models.Ticket.kind == kind)
    if status:
        query = query.filter(models.Ticket.status == status)
    return query.count()

In [56]:
x = get_tickets_count()

In [54]:
x

6782